# Advanced Regular Expression
Dalam dunia pengolahan data dan analisis teks, Ekspresi Reguler (regular expression atau regex) adalah pisau serbaguna yang dapat memotong, mengekstrak, memvalidasi, bahkan merangkai ulang potongan informasi dari data mentah. Banyak orang mengenalnya hanya sebagai alat pencarian pola sederhana, namun kenyataannya regex memiliki lapisan kemampuan yang jauh lebih dalam, mulai dari lookaround assertions, named groups, hingga pencocokan bersarang dengan rekursi.

# Persiapan Library dan Dataset

## Import library

In [42]:
import re
import regex   
import pandas as pd
from datetime import datetime
from dateutil import parser as dateparser
import unicodedata

## Dataset Contoh

In [43]:
server_logs = '''127.0.0.1 - - [10/Oct/2023:13:55:36 +0700] "GET /index.html HTTP/1.1" 200 1024 "-" "Mozilla/5.0"
192.168.1.5 - john [11/Oct/2023:08:20:10 +0700] "POST /api/v1/login HTTP/1.1" 500 512 "-" "curl/7.64"
ERROR [2023-10-11 08:20:11] Traceback (most recent call last):\n  File "app.py", line 23, in <module>\n    main()\n'''

mixed_notes = '''Contact: Andra <andra@example.com>, tel:+62-812-3456-7890. Invoice #INV-2023/09/0001 Rp 1.250.000. Due 2023-10-20.
Big sale: 50% off! Visit https://shop.example.com/deals?id=123&src=email
DOB: 12 Aug 1990, alternative: 1990/08/12'''

html_snippets = '''<div><p>Hello <span>World <b>bold</b></span></p><div>Nested <span>deep <i>italic</i></span></div></div>'''

# Parsing Log Server
Menangkap pola IP address, timestamp, status code, dan endpoint dari log server Apache/Nginx yang memiliki format tidak selalu konsisten. Menggunakan named groups untuk langsung memetakan hasil ekstraksi.

In [44]:
log_pattern = re.compile(r'''
    (?P<ip>\d{1,3}(?:\.\d{1,3}){3})      # IP address
    \s+-\s+                              # tanda " - "
    (?P<user>[^\s\[]+)                   # user (tidak mengandung spasi atau '[')
    \s+\[(?P<timestamp>[^\]]+)\]         # timestamp di dalam [ ]
    \s+"(?P<method>GET|POST|PUT|DELETE|PATCH|OPTIONS)  # method HTTP
    \s+(?P<path>[^\s]+)                  # path URL
    \s+(?P<proto>HTTP/\d\.\d)"           # protocol
    \s+(?P<status>\d{3})                 # status code (3 digit)
    \s+(?P<size>\d+|-)                   # ukuran respons (angka atau '-')
    \s+"(?P<ref>.*?)"                    # referer di dalam ""
    \s+"(?P<agent>.*?)"                  # user-agent di dalam ""
    ''', re.VERBOSE)

rows = []
for m in log_pattern.finditer(server_logs):
    d = m.groupdict()  # hasil regex jadi dictionary
    try:
        d['timestamp'] = dateparser.parse(d['timestamp'])  # ubah ke datetime
    except Exception:
        d['timestamp'] = d['timestamp']  # kalau gagal, biarkan string asli
    rows.append(d)


df_logs = pd.DataFrame(rows)
print(df_logs)

            ip  user                   timestamp method           path  \
0    127.0.0.1     -  10/Oct/2023:13:55:36 +0700    GET    /index.html   
1  192.168.1.5  john  11/Oct/2023:08:20:10 +0700   POST  /api/v1/login   

      proto status  size ref        agent  
0  HTTP/1.1    200  1024   -  Mozilla/5.0  
1  HTTP/1.1    500   512   -    curl/7.64  


Makna:
* ip → alamat klien.
* user → nama user (atau - kalau tidak ada).
* timestamp → waktu request.
* method → metode HTTP (GET, POST).
* path → jalur URL yang diminta.
* proto → protokol HTTP.
* status → kode status server.
* size → ukuran respons dalam byte.
* ref → referer (di sini - artinya kosong).
* agent → informasi aplikasi/alat yang digunakan klien.

# Advanced Text Cleaning and Normalization
Menghapus karakter tak terlihat (zero-width space, soft hyphen), mengatasi kombinasi Unicode, dan membersihkan teks dari alphanumeric noise dengan lookahead/lookbehind assertions.

In [45]:
def normalize_text(s):
    s = unicodedata.normalize('NFKC', s)
    s = re.sub('[\u200B\u200C\u200D\uFEFF]', '', s)
    s = s.replace('\u00AD', '')  # soft hyphen
    return s

print(normalize_text('Andra\u200Bexample — café'))

Andraexample — café


Keterangan:
* 'NFKC' → Normalization Form Compatibility Composition (merapikan bentuk & simbol menjadi bentuk paling standar).
* \u200B (zero-width space)
* \u200C (zero-width non-joiner)
* \u200D (zero-width joiner)
* \uFEFF (zero-width no-break space)
* s.replace('\u00AD', '') → menghapus soft hyphen (\u00AD), tanda sambung yang hanya muncul saat pemenggalan kata.

## Contoh lookaround sederhana

In [46]:
s = 'Halo. dunia. 3.14 not removed.'
s2 = re.sub(r'(?<=\w)\.(?=\s+[a-z])', '', s)
print(s2)

Halo dunia. 3.14 not removed.


Keterangan:
* Sebelumnya ada huruf/angka ((?<=\w)).
* Sesudahnya ada spasi diikuti huruf kecil ((?=\s+[a-z])).

# Ekstraksi Entitas Bernama tanpa Library NLP
Menggunakan regex untuk menemukan email, URL, nomor telepon, harga, dan tanggal dari teks mentah, memanfaatkan grup penamaan untuk struktur hasil yang rapi.

In [47]:
EMAIL = r'(?P<email>[\w.+-]+@[\w-]+\.[\w.-]+)'
URL = r'(?P<url>https?://[\w./?=&%#-]+)'
PHONE = r'(?P<phone>\+?\d{1,3}[\-\s]?\d{1,4}[\-\s]?\d{3,4}[\-\s]?\d{3,4})'
PRICE = r'(?P<price>Rp\s?[\d.,]+|\$\s?[\d,]+(?:\.\d{2})?)'
DATE = r'(?P<date>\b\d{1,2}[\-\s\/][A-Za-z0-9]{2,4}[\-\s\/]\d{2,4}\b|\b\d{4}-\d{2}-\d{2}\b)'

combined = re.compile('|'.join([EMAIL, URL, PHONE, PRICE, DATE]))

for m in combined.finditer(mixed_notes):
    print(m.groupdict())

{'email': 'andra@example.com', 'url': None, 'phone': None, 'price': None, 'date': None}
{'email': None, 'url': None, 'phone': '+62-812-3456-7890', 'price': None, 'date': None}
{'email': None, 'url': None, 'phone': None, 'price': 'Rp 1.250.000.', 'date': None}
{'email': None, 'url': None, 'phone': None, 'price': None, 'date': '2023-10-20'}
{'email': None, 'url': 'https://shop.example.com/deals?id=123&src=email', 'phone': None, 'price': None, 'date': None}
{'email': None, 'url': None, 'phone': None, 'price': None, 'date': '12 Aug 1990'}


Keterangan:
1. Email
   * [\w.+-]+ → bagian sebelum @ boleh huruf, angka, titik, plus, minus.
   * @ → pemisah.
   * [\w-]+ → nama domain.
   * \. → titik sebelum ekstensi domain.
   * [\w.-]+ → ekstensi domain (misalnya .com, .co.id).
2. URL
   * https?:// → http:// atau https://.
   * [\w./?=&%#-]+ → isi URL, boleh huruf, angka, titik, slash, query string, dsb. 
3. Phone
   * \+?\d{1,3} → kode negara opsional (+62).
   * [\-\s]? → boleh ada tanda minus atau spasi.
   * Lalu angka-angka dengan format umum nomor telepon.
4. Price
   * Bisa Rp atau $ diikuti angka.
   * [\d.,]+ → angka, titik, koma.
   * Bagian (?:\.\d{2})? untuk harga dalam dolar dengan dua angka desimal.
5. Date
   * Format DD-MM-YYYY, DD Mon YYYY, atau YYYY-MM-DD.
6. Kenapa banyak None?\
Karena hanya satu pola yang cocok di setiap match, sisanya otomatis kosong (None).
7. Untuk apa ini?\
Sangat berguna untuk ekstraksi entitas dari teks mentah (misalnya parsing email masuk, catatan meeting, atau teks bebas di web scraping).

# Pencocokan Pola Bersarang (rekursif dengan modul regex)
Menangani tanda kurung bersarang seperti pada parsing bahasa pemrograman, menggunakan modul regex Python yang mendukung pencocokan rekursif.

In [48]:
pattern = regex.compile(r'(?P<tag><(?P<name>\w+)(?:\s[^>]*)?>)(?:(?:(?R))|.*?)(?P<close></(?P=name)>)', flags=regex.DOTALL)

m = pattern.search(html_snippets)
if m:
    print('Matched block:')
    print(m.group(0))

Matched block:
<div><p>Hello <span>World <b>bold</b></span></p><div>Nested <span>deep <i>italic</i></span></div>


# Validasi password

In [49]:
def validate_password(pw, username=None):
    if len(pw) < 12:
        return False
    checks = [r'[A-Z]', r'[a-z]', r'\d', r'[^A-Za-z0-9]']
    if not all(re.search(c, pw) for c in checks):
        return False
    if username and username.lower() in pw.lower():
        return False
    return True

tests = [
    ('StrongPass!2023', 'andra'),
    ('Short1!', 'andra'),
    ('strongpassword2023', 'andra'),
    ('StrongPassword2023', 'andra'),
    ('StrongPass!word', 'andra'),
    ('StrongPass!2023', 'StrongPass!2023'),
    ('SuperSecure@99', 'andra'),
    ('Password!1234', 'pass'),
    ('Qwerty!Qwerty1', 'andra')
]

for pw, user in tests:
    print(pw, '->', validate_password(pw, username=user))

StrongPass!2023 -> True
Short1! -> False
strongpassword2023 -> False
StrongPassword2023 -> False
StrongPass!word -> False
StrongPass!2023 -> False
SuperSecure@99 -> True
Password!1234 -> False
Qwerty!Qwerty1 -> True


Keterangan:
| Password             | Username          | Alasan Valid / Tidak                                                                           |
| -------------------- | ----------------- | ---------------------------------------------------------------------------------------------- |
| `StrongPass!2023`    | `andra`           | ✅ **Valid** — panjang 15, ada huruf besar, kecil, angka, simbol, dan tidak mengandung "andra". |
| `Short1!`            | `andra`           | ❌ **Tidak valid** — kurang dari 12 karakter.                                                   |
| `strongpassword2023` | `andra`           | ❌ **Tidak valid** — tidak ada huruf besar, tidak ada simbol.                                   |
| `StrongPassword2023` | `andra`           | ❌ **Tidak valid** — tidak ada simbol.                                                          |
| `StrongPass!word`    | `andra`           | ❌ **Tidak valid** — tidak ada angka.                                                           |
| `StrongPass!2023`    | `StrongPass!2023` | ❌ **Tidak valid** — mengandung username yang sama persis.                                      |
| `SuperSecure@99`     | `andra`           | ✅ **Valid** — panjang 14, ada huruf besar, kecil, angka, simbol, tidak ada nama "andra".       |
| `Password!1234`      | `pass`            | ❌ **Tidak valid** — username `pass` ada di dalam password (case-insensitive).                  |
| `Qwerty!Qwerty1`     | `andra`           | ✅ **Valid** — panjang 15, semua kriteria terpenuhi.                                            |


# Check Luhn

In [50]:
cc_re = re.compile(r'(?P<number>(?:\d[ -]*?){13,19})') # Mencari nomor kartu kredit dengan panjang total digit 13 hingga 19 digit dan boleh dipisah "-"

def luhn_check(number):
    digits = [int(d) for d in re.sub(r'[^0-9]', '', number)][::-1]
    total = 0
    for i,d in enumerate(digits):
        if i % 2 == 1:
            d = d*2
            if d>9: d-=9
        total += d
    return total % 10 == 0

cards = [
    '4111 1111 1111 1111',
    '5500-0000-0000-0004',
    '378282246310005',
    '6011111111111117',
    '4111 1111 1111 1112',
    '1234 5678 9012 3456',
    '4111 1111 111',
    '4111 1111 1111 1111 1111'
]

for c in cards:
    found = cc_re.search(c)
    if found:
        print(c, '->', luhn_check(found.group('number')))
    else:
        print(c, '->', 'Format tidak cocok')

4111 1111 1111 1111 -> True
5500-0000-0000-0004 -> True
378282246310005 -> True
6011111111111117 -> True
4111 1111 1111 1112 -> False
1234 5678 9012 3456 -> False
4111 1111 111 -> Format tidak cocok
4111 1111 1111 1111 1111 -> True


Keterangan:
| Nomor Kartu                | Valid / Tidak | Alasan                                                 |
| -------------------------- | ------------- | ------------------------------------------------------ |
| `4111 1111 1111 1111`      | ✅ Valid       | Nomor uji coba Visa resmi, lolos Luhn.                 |
| `5500-0000-0000-0004`      | ✅ Valid       | Nomor uji coba Mastercard resmi, lolos Luhn.           |
| `378282246310005`          | ✅ Valid       | Nomor uji coba American Express, 15 digit, lolos Luhn. |
| `6011111111111117`         | ✅ Valid       | Nomor uji coba Discover Card, lolos Luhn.              |
| `4111 1111 1111 1112`      | ❌ Tidak valid | Gagal perhitungan Luhn (digit terakhir dimodifikasi).  |
| `1234 5678 9012 3456`      | ❌ Tidak valid | Tidak memenuhi aturan Luhn.                            |
| `4111 1111 111`            | ❌ Tidak valid | Hanya 11 digit, kurang dari 13 digit minimal.          |
| `4111 1111 1111 1111 1111` | ❌ Tidak valid | Lebih dari 19 digit.                                   |


# Regex + Pandas untuk Data Wrangling
Memecah kolom teks yang berisi gabungan nama, nomor faktur, dan tanggal menjadi kolom-kolom terpisah secara otomatis menggunakan regex di Pandas.

In [51]:
data = pd.DataFrame({'note':["Name: Andra; INV: INV-001; Date: 2023-10-11", "Name: Budi; INV: INV-002; Date: 2023/10/12"]})
pattern = r'Name:\s*(?P<name>[^;]+);\s*INV:\s*(?P<inv>[^;]+);\s*Date:\s*(?P<date>[^;]+)'
extracted = data['note'].str.extract(pattern)
extracted['date_norm'] = extracted['date'].apply(lambda x: dateparser.parse(x).date())
result = pd.concat([data, extracted], axis=1)
print(data)
print("-----------------")
print(result)

                                          note
0  Name: Andra; INV: INV-001; Date: 2023-10-11
1   Name: Budi; INV: INV-002; Date: 2023/10/12
-----------------
                                          note   name      inv        date  \
0  Name: Andra; INV: INV-001; Date: 2023-10-11  Andra  INV-001  2023-10-11   
1   Name: Budi; INV: INV-002; Date: 2023/10/12   Budi  INV-002  2023/10/12   

    date_norm  
0  2023-10-11  
1  2023-10-12  


# Redaksi Teks / Masking Data Sensitif
Mengaburkan informasi seperti email, nomor KTP, atau nomor rekening secara otomatis dengan regex replace, serta menampilkan perbandingan sebelum-sesudah.

In [52]:
# mask email
s = 'andra@example.com'
masked = re.sub(r'(?P<user>[^@]{1,3})([^@]*)(?=@)', lambda m: m.group('user') + '*'*len(m.group(2)), s)
print(masked)

# mask telepon: simpan kode negara + 2 digit terakhir
s = '+62-812-3456-7890'
masked_phone = re.sub(r'(\+?\d{1,3}[\-\s]?)([\d\-\s]+)(\d{2})$', lambda m: m.group(1) + re.sub(r'\d','*',m.group(2)) + m.group(3), s)
print(masked_phone)

and**@example.com
+62-***-****-**90


Keterangan mask email: simpan 1–3 huruf pertama sebelum @, sisanya diganti *
* (?P<user>[^@]{1,3}) → Tangkap 1–3 karakter pertama sebelum tanda @ (nama bagian awal email).
* ([^@]*) → Tangkap semua karakter lain sebelum tanda @ (bagian username yang akan dimask).
* (?=@) → Lookahead untuk memastikan kita berhenti tepat sebelum @ tanpa menghapusnya.
* m.group('user') → Ambil bagian depan username (1–3 huruf).
* '*' * len(m.group(2)) → Ganti sisa username dengan bintang sesuai panjangnya.
  
Keterangan mask telepon: simpan kode negara dan 2 digit terakhir, sisanya diganti *
* (\+?\d{1,3}[\-\s]?) → Tangkap kode negara (misalnya +62-).
* ([\d\-\s]+) → Tangkap angka-angka di tengah (akan dimask semua).
* (\d{2})$ → Tangkap 2 digit terakhir (disimpan asli).
* m.group(1) → Kode negara tetap ditampilkan.
* re.sub(r'\d', '*', m.group(2)) → Semua angka di tengah diganti bintang, tanda - tetap.
* m.group(3) → Dua digit terakhir tetap ditampilkan.

# Parsing HTML/XML sederhana (rekursif)
Menggunakan fitur rekursi pada modul regex untuk menemukan pasangan tag HTML/XML bersarang beserta kontennya, tanpa bantuan parser eksternal

In [53]:
html = '<div><p>Para <b>Pengunjung</b></p><div>Bagian <span>Dalam</span></div></div>'
expr = regex.compile(r'(?P<node><(?P<tag>\w+)(?:\s[^>]*)?>)\s*(?P<content>(?:(?R)|.*?))\s*(?P<close></(?P=tag)>)', flags=regex.DOTALL)
print(html)
print("-"*5)
print(expr)
print("-"*5)
for m in expr.finditer(html):
    print('TAG:', m.group('tag'))
    print('CONTENT:', m.group('content'))
    print('CLOSE:', m.group('close'))

<div><p>Para <b>Pengunjung</b></p><div>Bagian <span>Dalam</span></div></div>
-----
regex.Regex('(?P<node><(?P<tag>\\w+)(?:\\s[^>]*)?>)\\s*(?P<content>(?:(?R)|.*?))\\s*(?P<close></(?P=tag)>)', flags=regex.S | regex.V0)
-----
TAG: div
CONTENT: <p>Para <b>Pengunjung</b></p><div>Bagian <span>Dalam</span>
CLOSE: </div>


# Suite Pengujian & Benchmark Kinerja

In [54]:
import time
pat = regex.compile(r'^(?:[a-zA-Z0-9._%+-]+)@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
emails = ['user{}@example.com'.format(i) for i in range(10000)]
start = time.time()
for e in emails:
    pat.match(e)
print('Waktu (detik):', time.time()-start)

Waktu (detik): 0.019213438034057617


# Thank You